In [1]:
from pathlib import Path
import json
import numpy as np
import pandas as pd

import joblib

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    classification_report, confusion_matrix,
    precision_recall_curve
)

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

RANDOM_STATE = 42

DATA_DIR = Path("../data")
DATA_SAMPLE_DIR = Path("../data_sample")
ART_DIR = Path("../artifacts")
DATA_SAMPLE_DIR.mkdir(parents=True, exist_ok=True)
ART_DIR.mkdir(parents=True, exist_ok=True)

ACCEPTED_CSV = DATA_DIR / "accepted_2007_to_2018Q4.csv"
STAGE2_PARQUET = DATA_SAMPLE_DIR / "stage2_sample.parquet"

MODEL_PATH = ART_DIR / "stage2_pipeline.pkl"
METRICS_PATH = ART_DIR / "stage2_metrics.json"
UI_META_PATH = ART_DIR / "stage2_ui_metadata.json"


In [2]:
CANONICAL_COLS = [
    "loan_amount",
    "term",
    "purpose",
    "annual_income",
    "emp_length",
    "dti",
    "utilization",
    "delinquencies",
    "fico_est",
    "fico_missing",
    "emp_length_missing",
]
TARGET = "is_default"  # Stage 2 label: 1=bad(default), 0=good(paid)


In [3]:
def to_float(s: pd.Series) -> pd.Series:
    return pd.to_numeric(s, errors="coerce")

def pct_to_float(s: pd.Series) -> pd.Series:
    # handles "55.2%" or numeric
    if s.dtype == "O":
        s = s.astype(str).str.replace("%", "", regex=False).str.strip()
    return pd.to_numeric(s, errors="coerce")

def clean_emp_length_fast(s: pd.Series) -> pd.Series:
    if s.dtype != "O":
        return pd.to_numeric(s, errors="coerce")

    st = s.astype(str).str.lower()
    out = pd.Series(np.nan, index=s.index, dtype="float32")

    out[st.str.contains(r"<\s*1", na=False)] = 0.5
    out[st.str.contains(r"10\+", na=False)] = 10.0

    extracted = st.str.extract(r"(\d+)", expand=False)
    extracted_num = pd.to_numeric(extracted, errors="coerce")

    out = out.fillna(extracted_num)
    return out.astype("float32")

def clip_fico(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    return s.clip(lower=300, upper=850)

def clip_nonneg(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    return s.clip(lower=0)

def parse_term_months(s: pd.Series) -> pd.Series:
    if pd.api.types.is_numeric_dtype(s):
        return pd.to_numeric(s, errors="coerce")
    #extract first number found
    extracted = s.astype(str).str.extract(r"(\d+)", expand=False)
    return pd.to_numeric(extracted, errors="coerce")


In [ ]:
USECOLS = [
    "loan_amnt", "term", "purpose", "annual_inc", "emp_length", "dti",
    "revol_util", "delinq_2yrs", "fico_range_low", "fico_range_high",
    "loan_status"
]

acc = pd.read_csv(ACCEPTED_CSV, usecols=USECOLS, low_memory=False)
print("Loaded accepted shape:", acc.shape)

#Build target
good_status = {
    "Fully Paid",
    "Does not meet the credit policy. Status:Fully Paid"
}
bad_status = {
    "Charged Off",
    "Default",
    "Does not meet the credit policy. Status:Charged Off"
}

acc = acc[acc["loan_status"].isin(good_status | bad_status)].copy()
acc[TARGET] = acc["loan_status"].isin(bad_status).astype("int8")

print("Filtered to final-outcome loans:", acc.shape)
print(acc["loan_status"].value_counts())
print("Default rate:", float(acc[TARGET].mean()))

#Canonicalize features
df2 = pd.DataFrame({
    "loan_amount": to_float(acc["loan_amnt"]),
    "term": parse_term_months(acc["term"]),  
    "purpose": acc["purpose"].astype(str),
    "annual_income": to_float(acc["annual_inc"]),
    "emp_length": clean_emp_length_fast(acc["emp_length"]),
    "dti": pct_to_float(acc["dti"]),
    "utilization": pct_to_float(acc["revol_util"]),
    "delinquencies": to_float(acc["delinq_2yrs"]),
    "fico_est": (to_float(acc["fico_range_low"]) + to_float(acc["fico_range_high"])) / 2.0,
    TARGET: acc[TARGET].astype("int8")
})

#If term came in like " 36 months" fix it:
if df2["term"].dtype == "O":
    df2["term"] = pd.to_numeric(df2["term"].astype(str).str.extract(r"(\d+)", expand=False), errors="coerce")

df2["fico_est"] = clip_fico(df2["fico_est"])

#Missingness flags BEFORE fill
df2["fico_missing"] = df2["fico_est"].isna().astype("int8")
df2["emp_length_missing"] = df2["emp_length"].isna().astype("int8")

#Fill numeric with medians
for col in ["loan_amount","term","annual_income","emp_length","dti","utilization","delinquencies","fico_est"]:
    med = float(pd.to_numeric(df2[col], errors="coerce").median(skipna=True))
    df2[col] = pd.to_numeric(df2[col], errors="coerce").fillna(med)

#Sanity clips to  remove outliers
df2["loan_amount"] = clip_nonneg(df2["loan_amount"])
df2["annual_income"] = clip_nonneg(df2["annual_income"])
df2["delinquencies"] = clip_nonneg(df2["delinquencies"]).clip(upper=50)
df2["dti"] = df2["dti"].clip(lower=0, upper=80)
df2["utilization"] = df2["utilization"].clip(lower=0, upper=100)
df2["emp_length"] = df2["emp_length"].clip(lower=0, upper=50)
df2["term"] = df2["term"].clip(lower=0, upper=120)

print("Stage2 canonical shape:", df2.shape)
print("Any NaNs left?", df2.isna().any().any())

#Save Parquet
df2.to_parquet(STAGE2_PARQUET, index=False)
print("✅ Saved Stage 2 Parquet:", STAGE2_PARQUET)



Loaded accepted shape: (2260701, 11)
Filtered to final-outcome loans: (1348099, 12)
loan_status
Fully Paid                                             1076751
Charged Off                                             268559
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: count, dtype: int64
Default rate: 0.19980728418313493
Stage2 canonical shape: (1348099, 12)
Any NaNs left? False
✅ Saved Stage 2 Parquet: ..\data_sample\stage2_sample.parquet
term dtype: int64
term NaN rate: 0.0
term value counts:
term
36    1023206
60     324893
Name: count, dtype: int64


In [5]:
df2 = pd.read_parquet(STAGE2_PARQUET)
print(df2.shape)
print("Default rate:", float(df2[TARGET].mean()))
df2.head()


(1348099, 12)
Default rate: 0.19980728418313493


,loan_amount,term,purpose,annual_income,emp_length,dti,utilization,delinquencies,fico_est,is_default,fico_missing,emp_length_missing
0,3600.0,36,debt_consolidation,55000.0,10.0,5.91,29.7,0.0,677.0,0,0,0
1,24700.0,36,small_business,65000.0,10.0,16.06,19.2,1.0,717.0,0,0,0
2,20000.0,60,home_improvement,63000.0,10.0,10.78,56.2,0.0,697.0,0,0,0
3,10400.0,60,major_purchase,104433.0,3.0,25.37,64.5,1.0,697.0,0,0,0
4,11950.0,36,debt_consolidation,34000.0,4.0,10.20,68.4,0.0,692.0,0,0,0


In [6]:
X = df2.drop(columns=[TARGET]).copy()
y = df2[TARGET].astype(int).copy()

num_cols = X.select_dtypes(include=["number", "bool"]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

print("Numeric:", num_cols)
print("Categorical:", cat_cols)

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=RANDOM_STATE, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=RANDOM_STATE, stratify=y_temp
)

print("Train/Val/Test:", X_train.shape, X_val.shape, X_test.shape)
print("Train default rate:", float(y_train.mean()))


Numeric: ['loan_amount', 'term', 'annual_income', 'emp_length', 'dti', 'utilization', 'delinquencies', 'fico_est', 'fico_missing', 'emp_length_missing']
Categorical: ['purpose']
Train/Val/Test: (943669, 11) (202215, 11) (202215, 11)
Train default rate: 0.19980734770348502


In [7]:
numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols),
    ],
    remainder="drop"
)

clf = LogisticRegression(
    max_iter=800,
    class_weight="balanced",
    n_jobs=-1
)

pipe = Pipeline(steps=[
    ("prep", preprocess),
    ("model", clf)
])

pipe.fit(X_train, y_train)

val_proba = pipe.predict_proba(X_val)[:, 1]
val_auc = roc_auc_score(y_val, val_proba)
val_ap = average_precision_score(y_val, val_proba)

print("Val ROC-AUC:", val_auc)
print("Val PR-AUC :", val_ap)


C:\Users\john_\AppData\Roaming\Python\Python314\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


Val ROC-AUC: 0.678448382047025
Val PR-AUC : 0.33953328641684877


In [8]:
prec, rec, thr = precision_recall_curve(y_val, val_proba)
f1 = (2 * prec * rec) / (prec + rec + 1e-12)

best_idx = int(np.argmax(f1))
best_threshold = float(thr[best_idx]) if best_idx < len(thr) else 0.5

print("Best threshold (val, max F1):", best_threshold)
print("Best F1:", float(f1[best_idx]))


Best threshold (val, max F1): 0.49945026156178474
Best F1: 0.40469573629067956


In [9]:
test_proba = pipe.predict_proba(X_test)[:, 1]
test_auc = roc_auc_score(y_test, test_proba)
test_ap = average_precision_score(y_test, test_proba)

test_pred = (test_proba >= best_threshold).astype(int)

cm = confusion_matrix(y_test, test_pred)
report = classification_report(y_test, test_pred, digits=4)

print("Test ROC-AUC:", test_auc)
print("Test PR-AUC :", test_ap)
print("\nConfusion Matrix:\n", cm)
print("\nClassification Report:\n", report)


Test ROC-AUC: 0.6795873126870401
Test PR-AUC : 0.34120090585643514

Confusion Matrix:
 [[105125  56686]
 [ 15926  24478]]

Classification Report:
               precision    recall  f1-score   support

           0     0.8684    0.6497    0.7433    161811
           1     0.3016    0.6058    0.4027     40404

    accuracy                         0.6409    202215
   macro avg     0.5850    0.6278    0.5730    202215
weighted avg     0.7552    0.6409    0.6752    202215



In [10]:
from sklearn.ensemble import HistGradientBoostingClassifier

gb = HistGradientBoostingClassifier(
    learning_rate=0.05,
    max_depth=6,
    max_iter=300,
    random_state=RANDOM_STATE
)

gb_pipe = Pipeline(steps=[
    ("prep", preprocess),
    ("model", gb)
])

gb_pipe.fit(X_train, y_train)

val_proba_gb = gb_pipe.predict_proba(X_val)[:, 1]
val_auc_gb = roc_auc_score(y_val, val_proba_gb)
val_ap_gb = average_precision_score(y_val, val_proba_gb)

print("GB Val ROC-AUC:", val_auc_gb)
print("GB Val PR-AUC :", val_ap_gb)


GB Val ROC-AUC: 0.6874405278446104
GB Val PR-AUC : 0.3514251478699275


In [11]:
test_proba_gb = gb_pipe.predict_proba(X_test)[:, 1]
test_auc_gb = roc_auc_score(y_test, test_proba_gb)
test_ap_gb = average_precision_score(y_test, test_proba_gb)

print("GB Test ROC-AUC:", test_auc_gb)
print("GB Test PR-AUC :", test_ap_gb)


GB Test ROC-AUC: 0.6887419428230931
GB Test PR-AUC : 0.3525630074473265


In [12]:
from sklearn.metrics import precision_recall_curve

target_precision = 0.50  #favor precision

prec, rec, thr = precision_recall_curve(y_val, val_proba_gb)

valid = np.where(prec[:-1] >= target_precision)[0]

if len(valid) == 0:
    best_idx = int(np.argmax(prec[:-1]))
    chosen_threshold = float(thr[best_idx])
    chosen_note = f"Target precision {target_precision} not reachable; using max precision point."
else:
    best_idx = valid[np.argmax(rec[:-1][valid])]
    chosen_threshold = float(thr[best_idx])
    chosen_note = f"Chosen to meet precision >= {target_precision} with max recall."

print("Chosen threshold:", chosen_threshold)
print("Note:", chosen_note)
print("Val precision @ threshold:", float(prec[best_idx]))
print("Val recall    @ threshold:", float(rec[best_idx]))


Chosen threshold: 0.4465711966985147
Note: Chosen to meet precision >= 0.5 with max recall.
Val precision @ threshold: 0.5
Val recall    @ threshold: 0.08375408375408376


In [13]:
from sklearn.metrics import confusion_matrix, classification_report

test_pred = (test_proba_gb >= chosen_threshold).astype(int)

cm = confusion_matrix(y_test, test_pred)
tn, fp, fn, tp = cm.ravel()

precision = tp / (tp + fp + 1e-12)
recall = tp / (tp + fn + 1e-12)
f1 = 2 * precision * recall / (precision + recall + 1e-12)
fpr = fp / (fp + tn + 1e-12)
pass_rate = (tp + fp) / (tp + fp + tn + fn)

print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", classification_report(y_test, test_pred, digits=4))

print("\nOperational metrics (test):")
print(f"Precision (bad=1): {precision:.4f}")
print(f"Recall (bad=1):    {recall:.4f}")
print(f"F1 (bad=1):        {f1:.4f}")
print(f"FPR:               {fpr:.4f}")
print(f"Flag rate (pred=1):{pass_rate:.4f}")


Confusion Matrix:
 [[158428   3383]
 [ 37022   3382]]

Classification Report:
               precision    recall  f1-score   support

           0     0.8106    0.9791    0.8869    161811
           1     0.4999    0.0837    0.1434     40404

    accuracy                         0.8002    202215
   macro avg     0.6553    0.5314    0.5152    202215
weighted avg     0.7485    0.8002    0.7383    202215


Operational metrics (test):
Precision (bad=1): 0.4999
Recall (bad=1):    0.0837
F1 (bad=1):        0.1434
FPR:               0.0209
Flag rate (pred=1):0.0335


In [14]:

def threshold_for_min_precision(y_true, proba, target_precision: float):
    prec, rec, thr = precision_recall_curve(y_true, proba)
    valid = np.where(prec[:-1] >= target_precision)[0]
    if len(valid) == 0:
        best_idx = int(np.argmax(prec[:-1]))
        return float(thr[best_idx]), float(prec[best_idx]), float(rec[best_idx]), "max_precision_fallback"
    best_idx = valid[np.argmax(rec[:-1][valid])]
    return float(thr[best_idx]), float(prec[best_idx]), float(rec[best_idx]), "min_precision_max_recall"

t_high, p_high, r_high, rule_high = threshold_for_min_precision(y_val, val_proba_gb, 0.50)
t_med,  p_med,  r_med,  rule_med  = threshold_for_min_precision(y_val, val_proba_gb, 0.35)

print("HIGH threshold:", t_high, "| val precision:", p_high, "| val recall:", r_high, "|", rule_high)
print("MED  threshold:", t_med,  "| val precision:", p_med,  "| val recall:", r_med,  "|", rule_med)


HIGH threshold: 0.4465711966985147 | val precision: 0.5 | val recall: 0.08375408375408376 | min_precision_max_recall
MED  threshold: 0.2560286357072218 | val precision: 0.3500009720629119 | val recall: 0.4455746955746956 | min_precision_max_recall


In [15]:
def risk_band(proba, t_med, t_high):
    if proba >= t_high:
        return "High"
    elif proba >= t_med:
        return "Medium"
    else:
        return "Low"

bands = pd.Series([risk_band(p, t_med, t_high) for p in test_proba_gb])
print(bands.value_counts(normalize=True))

# Check actual default rates by band (this is VERY compelling for your UI + writeup)
band_df = pd.DataFrame({"band": bands, "y": y_test.values})
display(band_df.groupby("band")["y"].mean().sort_index())


Low       0.745073
Medium    0.221472
High      0.033454
Name: proportion, dtype: float64


band
High      0.499926
Low       0.147931
Medium    0.328994
Name: y, dtype: float64

In [ ]:
joblib.dump(pipe, MODEL_PATH)
print("Saved model:", MODEL_PATH)

metrics = {
    "stage": "stage2_default_risk",
    "model": "hist_gradient_boosting",
    "target_definition": {
        "good": ["Fully Paid", "Does not meet the credit policy. Status:Fully Paid"],
        "bad": ["Charged Off", "Default", "Does not meet the credit policy. Status:Charged Off"],
        "dropped": ["Current", "Late (16-30 days)", "Late (31-120 days)", "In Grace Period"]
    },
    "features": X.columns.tolist(),
    "val_roc_auc": float(val_auc),
    "val_pr_auc": float(val_ap),
    "test_roc_auc": float(test_auc),
    "test_pr_auc": float(test_ap),

    "threshold": float(t_high),

    "band_policy": {
        "risk_bands": {
            "low":    {"max": float(t_med)},
            "medium": {"min": float(t_med), "max": float(t_high)},
            "high":   {"min": float(t_high)}
        },
        "how_chosen": {
            "t_high": {"min_precision": 0.50, "val_precision": float(p_high), "val_recall": float(r_high), "rule": rule_high},
            "t_med":  {"min_precision": 0.35, "val_precision": float(p_med),  "val_recall": float(r_med),  "rule": rule_med}
        }
    },

    "confusion_matrix": cm.tolist(),
    "classification_report": report
}

with open(METRICS_PATH, "w") as f:
    json.dump(metrics, f, indent=2)

print("Saved metrics:", METRICS_PATH)

def make_numeric_metadata(df_train: pd.DataFrame, columns: list[str]) -> dict:
    meta = {}
    for c in columns:
        s = pd.to_numeric(df_train[c], errors="coerce").dropna()
        if len(s) == 0:
            continue
        meta[c] = {
            "min": float(s.min()),
            "max": float(s.max()),
            "p1": float(np.percentile(s, 1)),
            "p99": float(np.percentile(s, 99)),
            "recommended_min": float(np.percentile(s, 10)),
            "recommended_max": float(np.percentile(s, 90)),
        }
    return meta

ui_meta = {
    "stage": "stage2_default_risk",
    "numeric": make_numeric_metadata(X_train, num_cols),
    "categorical": {
        c: X_train[c].astype(str).value_counts().head(30).index.tolist()
        for c in cat_cols
    }
}

with open(UI_META_PATH, "w") as f:
    json.dump(ui_meta, f, indent=2)

print("Saved UI metadata:", UI_META_PATH)

print("\nThresholds:")
print("t_med :", t_med,  "| val precision:", p_med,  "| val recall:", r_med)
print("t_high:", t_high, "| val precision:", p_high, "| val recall:", r_high)

Saved model: ..\artifacts\stage2_pipeline.pkl
Saved metrics: ..\artifacts\stage2_metrics.json
Saved UI metadata: ..\artifacts\stage2_ui_metadata.json

Thresholds:
t_med : 0.2560286357072218 | val precision: 0.3500009720629119 | val recall: 0.4455746955746956
t_high: 0.4465711966985147 | val precision: 0.5 | val recall: 0.08375408375408376
